In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import astropy
import os, sys
import pandas as pd
import numpy as np
from astropy.table import Table, join
import desispec

# If not using the desiconda version of prospect: EDIT THIS to your path
sys.path.insert(0,"/global/homes/b/bid13/DESI/DESI-II/prospect/py/") 
from prospect import viewer, utilities

In [ ]:
data_path = Path("/global/cfs/cdirs/desi/users/bid13/DESI_II/pilot_obs/COSMOS/LSST_WLY1")

In [ ]:
all_spectra = desispec.io.read_spectra(data_path / "spectra.fits")


all_redrock = Table.read(data_path / "zbest.fits")
all_redrock[all_redrock["SUBTYPE"].mask]["SUBTYPE"] = b'NA'
all_redrock["SUBTYPE"].mask =False

for c in all_redrock.columns:
    # print(c)
    if isinstance(all_redrock[c], astropy.table.column.MaskedColumn):
        all_redrock[c] = all_redrock[c].value.data
all_redrock.add_index("TARGETID")

rr_details = Table.read(data_path / "redrock_details.fits")
for c in rr_details.columns:
    if c == "SPECTYPE":
        rr_details[c] = rr_details[c].value.astype(str)
    if c =="SUBTYPE":
        rr_details[c] = rr_details[c].value.data.astype(str)
rr_details.add_index("TARGETID")

cat = Table.read(data_path / "merged_cat.fits")
cat.sort("DELTACHI2")
cat.reverse()
cat.add_index("TARGETID")




### Create one single page

In [ ]:
sel_mask = (cat["EXPTIME"]/3600>4.5)


In [ ]:
sel_targets = cat["TARGETID"][sel_mask].value
sel_spectra = all_spectra.select(targets = sel_targets)

# sel_spectra.fibermap = join(sel_spectra.fibermap, cat[['mag_i','mag_i_fiber',"TARGETID"]],keys="TARGETID",join_type="left")
sel_spectra.fibermap['mag_i'] = cat.loc[sel_spectra.fibermap['TARGETID']]['mag_i']
sel_spectra.fibermap['mag_i_fiber'] =  cat.loc[sel_spectra.fibermap['TARGETID']]['mag_i_fiber']
sel_targets = sel_spectra.fibermap["TARGETID"].value

sel_zcat = all_redrock.loc[sel_targets]


sel_rr_det = rr_details.loc[sel_targets]

In [ ]:
print(sel_spectra.num_spectra(),"spectra selected")

In [ ]:
# divide first n assign >1 then in chunks of deltachi2

In [ ]:
# Run this cell to have the VI tool !
viewer.plotspectra(sel_spectra, 
                   zcatalog=sel_zcat,
                   redrock_cat=sel_rr_det,
                   notebook=True, title='test_exptime_gr_7', 
                   html_dir = "/global/cfs/cdirs/desi/users/bid13/DESI_II/VI_pages"
                   # mask_type='SV1_DESI_TARGET'
                  )

### Generate all pages

In [ ]:
def create_vi_page(sel_mask,cat, file_name, dir_name ):
    
    
    all_targets = cat["TARGETID"][sel_mask].value
    for i in range(0,len(all_targets.data), 50):
        sel_targets = all_targets[i:i+50]
        sel_spectra = all_spectra.select(targets = sel_targets)
        sel_spectra.fibermap['mag_i'] = cat.loc[sel_spectra.fibermap['TARGETID']]['mag_i']
        sel_spectra.fibermap['mag_i_fiber'] =  cat.loc[sel_spectra.fibermap['TARGETID']]['mag_i_fiber']
        sel_targets = sel_spectra.fibermap["TARGETID"].value
        sel_zcat = all_redrock.loc[sel_targets]
        sel_rr_det = rr_details.loc[sel_targets]

        viewer.plotspectra(sel_spectra, 
                           zcatalog=sel_zcat,
                           redrock_cat=sel_rr_det,
                           notebook=False, title=f'{file_name}_{(i//50)+1}', 
                           html_dir = f"/global/cfs/cdirs/desi/users/bid13/DESI_II/VI_pages/COSMOS/{dir_name}"
                           # mask_type='SV1_DESI_TARGET'
                          )

In [ ]:
# only 1 fiber
mask = (cat["NASSIGN"] <2)
create_vi_page(mask,cat, "single_visit", "single_visit")
mask.sum()

In [ ]:
# more than 1 fiber and bright
mask = (cat["NASSIGN"] >1) & (cat["mag_i"]<=23)
create_vi_page(mask,cat, "i_leq_23", "i_less_than_23")
mask.sum()

In [ ]:
# more than 1 fiber and bright
mask = (cat["NASSIGN"] >1) & (cat["mag_i"]>=23)
create_vi_page(mask,cat, "i_geq_23", "i_greater_than_23")
mask.sum()

In [ ]:
# import matplotlib.pyplot as plt
# mask = (cat["NASSIGN"] >1) & (cat["mag_i"]>=23)
# all_targets = cat["TARGETID"][mask].value
# sel_spectra = all_spectra.select(targets = all_targets)

# plt.hist(sel_spectra.fibermap["mag_i"])

### Create VI pages for conflicted scores

In [ ]:
conflicted_cat = pd.read_csv(Path(os.environ["SCRATCH"]) / "DESI-II-VI-results" / "conflicted_cat.csv")

In [ ]:
sel_targets = np.unique(conflicted_cat["TARGETID"])
sel_spectra = all_spectra.select(targets = sel_targets)
sel_targets = sel_spectra.fibermap["TARGETID"].value

sel_zcat = all_redrock.loc[sel_targets]


sel_rr_det = rr_details.loc[sel_targets]

In [ ]:
print(sel_spectra.num_spectra(),"spectra selected")

In [ ]:
# Run this cell to have the VI tool !
viewer.plotspectra(sel_spectra, 
                   zcatalog=sel_zcat,
                   redrock_cat=sel_rr_det,
                   notebook=False, title='VI_conflicts', 
                   html_dir = "/global/cfs/cdirs/desi/users/bid13/DESI_II/VI_pages"
                   # mask_type='SV1_DESI_TARGET'
                  )